In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

from ostk.core.filesystem import Directory

from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Composite
from ostk.mathematics.geometry.d3.objects import Point

from ostk.physics import Environment
from ostk.physics.coordinate import Frame
from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel
from ostk.physics.environment.magnetic import Earth as EarthMagneticModel
from ostk.physics.environment.objects.celestial_bodies import Earth, Moon, Sun
from ostk.physics.time import DateTime, Duration, Instant, Scale, Time
from ostk.physics.units import Mass, Length

from ostk.astrodynamics.trajectory.state import NumericalSolver
from ostk.astrodynamics.event_condition import RealCondition
from ostk.astrodynamics.trajectory.orbit.models.brouwerLyddaneMean import (
    BrouwerLyddaneMeanShort
)
from ostk.astrodynamics.trajectory import StateBuilder
from ostk.astrodynamics.trajectory.state import CoordinatesBroker, CoordinatesSubset
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianPosition, CartesianVelocity
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.astrodynamics.trajectory import Orbit, Sequence
from ostk.astrodynamics import Dynamics
from ostk.astrodynamics.dynamics import Thruster
from ostk.astrodynamics.event_condition import LogicalCondition
from ostk.astrodynamics.flight.system import SatelliteSystem, PropulsionSystem
from ostk.astrodynamics.guidance_law import QLaw

## User inputs

In [ ]:
earth = Earth.from_models(
    EarthGravitationalModel(EarthGravitationalModel.Type.EGM96, Directory.undefined(), 20, 20),
    EarthMagneticModel(EarthMagneticModel.Type.Undefined),
    EarthAtmosphericModel(EarthAtmosphericModel.Type.NRLMSISE00),
)

environment = Environment(Instant.J2000(), [earth, Sun.default(), Moon.default()])

In [ ]:
instant = Instant.date_time(DateTime(2023,1,1), Scale.UTC)
orbit = Orbit.sun_synchronous(instant, Length.kilometers(580.0), Time.midnight(), Earth.default())
initial_state = orbit.get_state_at(instant)

In [ ]:
mass = Mass.kilograms(214.6)
wet_mass = Mass.kilograms(14.3)
propulsion_system = PropulsionSystem(thrust_si_unit=0.0141, specific_impulse_si_unit=1004.7)

## Setup environment, initial state and Satellite System

In [ ]:
satellite_geometry = Composite(Cuboid(
    Point(0.0, 0.0, 0.0), np.eye(3).tolist(), [1.0, 0.0, 0.0]
))
satellite_system = SatelliteSystem(mass, satellite_geometry, np.eye(3), 2.0, 2.2, propulsion_system)

In [ ]:
coordinates_broker = CoordinatesBroker(
    [
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinatesSubset.mass(),
        CoordinatesSubset.surface_area(),
        CoordinatesSubset.drag_coefficient(),
    ]
)

state_builder = StateBuilder(
    frame=Frame.GCRF(),
    coordinates_subsets=[
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinatesSubset.mass(),
        CoordinatesSubset.surface_area(),
        CoordinatesSubset.drag_coefficient(),
    ],
)

coordinates = [
    *initial_state.get_coordinates().tolist(),
    mass.in_kilograms() + wet_mass.in_kilograms(),
    satellite_system.get_cross_sectional_surface_area(),
    satellite_system.get_drag_coefficient()
]

initial_state = state_builder.build(initial_state.get_instant(), coordinates)

In [ ]:
dynamics = Dynamics.from_environment(environment)
numerical_solver = NumericalSolver.default_conditional()

### Sequence

In [ ]:
initial_coe = orbit.access_kepler_model().get_classical_orbital_elements()
target_coe = COE(
    initial_coe.get_semi_major_axis() + Length.kilometers(3.0),
    initial_coe.get_eccentricity(),
    initial_coe.get_inclination(),
    initial_coe.get_raan(),
    initial_coe.get_aop(),
    initial_coe.get_true_anomaly()
)
q_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLaw.Parameters(
        element_weights={
            COE.Element.SemiMajorAxis: (1.0, 50.0),
            COE.Element.Eccentricity: (1.0, 1e-4),
        }
    )
)

In [ ]:
sequence = Sequence(
    numerical_solver=numerical_solver,
    dynamics=dynamics,
    maximum_propagation_duration=Duration.days(30.0),
    verbosity=0
)

evaluator = lambda state: BrouwerLyddaneMeanShort.cartesian((state.get_position(), state.get_velocity()), EarthGravitationalModel.EGM2008.gravitational_parameter).get_semi_major_axis().in_meters()

# burn till at 500.0
sma_condition = RealCondition(
    name="Mean SMA crossing - thrust",
    criterion=RealCondition.Criterion.StrictlyPositive,
    evaluator=evaluator,
    target_value=evaluator(initial_state) + 3000.0,
)
duration_condition = RealCondition.duration_condition(
    criterion=RealCondition.Criterion.StrictlyPositive,
    duration=initial_coe.get_orbital_period(EarthGravitationalModel.EGM2008.gravitational_parameter)/2.0
)

event_condition = LogicalCondition(
    "maneuver condition",
    LogicalCondition.Type.Or,
    [sma_condition, duration_condition]
)
sequence.add_maneuver_segment(
    event_condition,
    Thruster(satellite_system, q_law)
)

# coast for 20.0 minutes
coast_condition = RealCondition.duration_condition(
    criterion=RealCondition.Criterion.StrictlyPositive,
    duration=Duration.minutes(20.0),
)
sequence.add_coast_segment(coast_condition)

In [ ]:
sol = sequence.solve_to_condition(initial_state, sma_condition)
states = sol.get_states()

In [ ]:
data = []
for state in states:
    blmshort = BrouwerLyddaneMeanShort.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "altitude": float(blmshort.get_semi_major_axis().in_kilometers() - earth.get_equatorial_radius().in_kilometers()),
            "mass": float(state.get_coordinates()[6]),
            "eccentricity": float(blmshort.get_eccentricity()),
            "periapsis": float(blmshort.get_periapsis_radius()),
            "apoapsis": float(blmshort.get_apoapsis_radius()),
            "time": state.get_instant().get_date_time(Scale.UTC),
            "x": state.get_coordinates()[0],
            "y": state.get_coordinates()[1],
            "z": state.get_coordinates()[2]
        }
    )
df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="altitude", color="mass", title=f"Altitude [km] (initial, target) ({float(evaluator(initial_state) - earth.get_equatorial_radius().in_meters()) / 1000.0:.4f}, {float(sma_condition.get_target().value - earth.get_equatorial_radius().in_meters()) / 1000.0:.4f})")
for segment_solution in sol.segment_solutions:
    time = segment_solution.states[-1].get_instant().get_date_time(Scale.UTC)
    fig.add_vline(
        x=time.isoformat()
    )
fig.show()

In [ ]:
fig = px.scatter(df, x="time", y="eccentricity", color="mass", title=f"Eccentricity")
for segment_solution in sol.segment_solutions:
    time = segment_solution.states[-1].get_instant().get_date_time(Scale.UTC)
    fig.add_vline(
        x=time.isoformat()
    )
fig.show()

In [ ]:
px.scatter(df, x="periapsis", y="apoapsis", color="eccentricity", title="Apoapsis vs Periapsis")